To Do:
    Fix save dataset save position position
    
    Separate cleaned data save(possibly move to helpfun?)
    
    Clean desc strings
    
    Split desc strings into single objects
    
    Make bag-of words
    
    Decide network architechure 
    
    Make network

In [ ]:
'''
Purpose: Classify transactions for budget record keeping
Resources: http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
            http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
            https://www.dotnetperls.com/translate-python
            https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words
            http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/
Strategy:
    Use basic CNN w/ bag-o-words word-vectorization to classify transaction entries based on bank statement description and pre-selected categories

Data: Raw CC transaction data (4/14/17 - 10/11/17) hand stripped and categorized from monthly base statements
 Raw(Call='dataset'):   
    Entries       200
    Cols          date, desc, blank, amount, c1, c2
    Classes       c2             10:
                                    A  -  23  Gas
                                    B  -  6   Bill
                                    B1 -  1   One-time bill
                                    C  -  4   Clothes
                                    D  -  18  R&D
                                    G  -  24  Grocerry
                                    M  -  28  MISC
                                    R  -  71  Restaurant
                                    V  -  13  Vidya
 Clean('df'):
    Entries       196
    Features      desc, c2       
    Classes       c2             8:
                                    A  -  23  Gas
                                    B  -  6   Bill
                                    D  -  18  R&D
                                    G  -  24  Grocerry
                                    M  -  28  MISC
                                    R  -  71  Restaurant
                                    V  -  13  Vidya
Python      v3.6.3
helperFun   v0.0.1
pandas      v0.22.0
re          v2.2.1
'''

In [1]:
import pandas as pd
import re
#clean strings
def c_str(strings, clean_list = []):
    '''
    Purpose: Clean string of punct and unwanted words, then split into lists
    
    :type text str
    :type clean_list list
    ********
    clean_list: optional arg
    Accepts passed list of string objs to be removed from string
    '''
    csstr = []
    dDesc = strings
    dDesc = dDesc.str.lower()
    replace_re = '|'.join([re.escape(s) for s in clean_list] + [r'\s+', r'[0-9]+'])
    for idx, str in enumerate(dDesc):
        cstr = []
        str = re.sub(r'\W+', ' ', str)
        if str.startswith('uber'):
            str = 'uber'
        str = str.split()
        for word in str:
            cstr += [re.sub(replace_re, '', word)]
        str = cstr
        str = ' '.join([word for word in str if word])
        dDesc[idx] = str
    return dDesc

In [2]:
####-------
#New transactions

df = pd.read_csv('./data/Budget - Sheet14.csv')
clean_list = ['phoenix','avondale','tempe', 'help', 'glendale','az','ny']
df.loc[df.c2 == 'B1','c2'] = 'B'

df.desc = c_str(df.desc, clean_list)
df.to_pickle('./data/Trans_04-27.pkl')
df.c2.value_counts()

R    175
A     69
D     51
G     45
B     43
S     41
M     38
V     29
Name: c2, dtype: int64

In [3]:
tdf = df.copy().drop('price',axis=1)
for clss in d_classes:
    if clss not in ['R']:
        t = tdf.loc[tdf.c2 == clss,:].desc.value_counts()
        t = list(zip(t.keys().tolist(),t.tolist()))
        targ = t[0][1]
        for desc, act in t[1:]:
            need = targ - act
            if need < targ:
                for i in range(need):
                    tdf = tdf.append({'desc':desc,'c2':clss},ignore_index=True)
tdf

NameError: name 'd_classes' is not defined

In [22]:
# def fraudy(s_targ = 2000, pklpath='./data/Trans_04-27.pkl', blklst=[]):
#     tdf = pd.read_pickle(pklpath).drop('price',axis=1)
#     labels, d_classes = pd.factorize(df.c2)
#     s_act = len(tdf)
#     s_need =  s_targ - s_act
#     if s_need < s_targ:
#         b_targ = s_need/len(d_classes)
#         for clss in d_classes:
#             t = tdf.loc[tdf.c2 == clss,:].desc.value_counts()
#             t = list(zip(t.keys().tolist(),t.tolist()))
#             for desc, b_act in t:
#                 b_targ = b_targ / len(t)
#                 b_need = b_targ - b_act
# #                 if b_need < b_targ:
# #                     print('Creating {} for class {}'.format(b_need, clss))
# #                     for i in range(need):
# #                         tdf = tdf.append({'desc':desc,'c2':clss},ignore_index=True)
#     return tdf

# fraudy()

Creating -1.455 for class B
Creating -3.6982 for class B
Creating -2.987928 for class B
Creating -1.99951712 for class B
Creating -1.9999806848 for class B
Creating -1.999999227392 for class B
Creating -1.99999996909568 for class B
Creating -1.9999999987638273 for class B
Creating -0.9999999999505531 for class B
Creating -0.9999999999980221 for class B
Creating -0.9999999999999208 for class B
Creating -0.9999999999999968 for class B
Creating -0.9999999999999999 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -1.0 for class B
Creating -22.0 for class S
Creating -15.0 for class S
Creating -2.0 for class S
Creating -1.0 for class S
Creating -1.0 for class S
Creating -43.0 for class A
Creating -5.0 for class A
Creating -5.0 for class A

KeyboardInterrupt: 

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten

def steks(d1_nodes=2048,
          d1_activation='relu',
          d1_dropout=.2,
          d2_nodes=1024,
          d2_activation='relu',
          d2_dropout=.2,
          d3_nodes=512,
          d3_activation='relu',
          d3_dropout=.2,
          d4_nodes=256,
          d4_activation='relu',
          d4_dropout=.2,
          d5_nodes=128,
          d5_activation='relu',
          d5_dropout=.2,
          d6_nodes=64,
          d6_activation='relu',
          d6_dropout=.2,
          lr=0.0006,
          epochs=50):
    
    # create model
#     model = Sequential()
#     model.add(Embedding(em_input_dim,em_nodes,input_length=313))
#     model.add(Flatten())
#     model.add(Dense(d1_nodes, input_shape=(313,),activation=d1_activation))
#     model.add(Dropout(.2))
#     model.add(Dense(d2_nodes, activation=d2_activation))
#     model.add(Dropout(.2))
#     model.add(Dense(output_dim, activation='softmax'))

    model = Sequential()
    model.add(Dense(d1_nodes, input_shape=(num_words,),activation=d1_activation))
    model.add(Dropout(d1_dropout))
#     model.add(Embedding(em_input_dim,em_nodes,input_length=200))
#     model.add(Flatten())
    model.add(Dense(d1_nodes, activation=d1_activation))
    model.add(Dropout(d1_dropout))
    model.add(Dense(d2_nodes, activation=d2_activation))
    model.add(Dropout(d2_dropout))
    model.add(Dense(d3_nodes, activation=d3_activation))
    model.add(Dropout(d3_dropout))
    model.add(Dense(d4_nodes, activation=d4_activation))
    model.add(Dropout(d4_dropout))
    model.add(Dense(d5_nodes, activation=d5_activation))
    model.add(Dropout(d5_dropout))
    model.add(Dense(d6_nodes, activation=d6_activation))
    model.add(Dropout(d6_dropout))
    
    model.add(Dense(output_dim, activation='softmax'))


    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=lr,decay=0.0000001), metrics=['accuracy'])

    model.fit(X_train, Y_train, epochs=epochs, batch_size=32, validation_data=(X_valid,Y_valid),shuffle=True)
#     score = [model.evaluate(X_valid,Y_valid)[1], model.evaluate(X_test,Y_test)[1]]
    score = model.evaluate(X_test,Y_test)[1]
    return score

scores = []
# for dn in [200,150]:
#     for dt in [150,100]:
#         for dh in [100,50]:
#             scores.append(('d1_nodes={}, d2_nodes={}, d3_nodes={}'.format(dn,dt,dh),steks(d4_nodes=dn,d5_nodes=dt,d6_nodes=dh)))
# for i in [0.0006,0.00056,0.00036]:
#     for e in [50,25,75]:
#         scores.append(('dropout = {}'.format(i),steks(lr=i,epochs=e)))
scores.append(steks())

Train on 343 samples, validate on 74 samples
Epoch 1/50
224/343 [==================>...........] - ETA: 2s - loss: 1.9546 - acc: 0.369 - ETA: 1s - loss: 1.8944 - acc: 0.3750

KeyboardInterrupt: 

In [ ]:
import numpy as np
fails = 0
successes = 0

for row in df[313:].itertuples():
    idx = row.Index
    actual = df.at[idx,'c2']
    pred = model.evaluate(tokenz.texts_to_matrix(row.desc))[0]
    pred = pred.argmax(axis=-1)
    if actual != pred:
        fails += 1
    elif actual == pred:
        successes += 1
        
    
print('Accuracy: {}'.format(round(successes/len(df[313:])*100,2)))

In [19]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import pandas as pd

def data():
    
    def c_str(strings, clean_list = []):
        '''
        Purpose: Clean string of punct and unwanted words, then split into lists

        :type text str
        :type clean_list list
        ********
        clean_list: optional arg
        Accepts passed list of string objs to be removed from string
        '''
        csstr = []
        dDesc = strings
        dDesc = dDesc.str.lower()
        replace_re = '|'.join([re.escape(s) for s in clean_list] + [r'\s+', r'[0-9]+'])
        for idx, str in enumerate(dDesc):
            cstr = []
            str = re.sub(r'\W+', ' ', str)
            if str.startswith('uber'):
                str = 'uber'
            str = str.split()
            for word in str:
                cstr += [re.sub(replace_re, '', word)]
            str = cstr
            str = ' '.join([word for word in str if word])
            dDesc[idx] = str
        return dDesc
    
    df = pd.read_csv('./data/Budget - Sheet14.csv')
    clean_list = ['phoenix','avondale','tempe', 'help', 'glendale','az','ny']
    df.loc[df.c2 == 'B1','c2'] = 'B'

    df.desc = c_str(df.desc, clean_list)
    df.to_pickle('./data/Trans_04-27.pkl')
    df.c2.value_counts()
    tdf = df.copy().drop('price',axis=1)
                        
    tokenz = Tokenizer(num_words=180)
    tokenz.fit_on_texts(df.desc)
    labels, d_classes = pd.factorize(df.c2)
    input_dim = len(tokenz.word_index)
    encoded = tokenz.texts_to_matrix(df.desc)

    labels = to_categorical(labels)

    output_dim = len(d_classes)

    train_perc = .7
    le = len(encoded)
    t_end = int(le * train_perc)
    v_end = int(t_end + (le - t_end) * .5)

    X_train = encoded[:t_end]
    X_valid = encoded[t_end:v_end]
    X_test = encoded[v_end:]

    Y_train = labels[:t_end]
    Y_valid = labels[t_end:v_end]
    Y_test = labels[v_end:]

    return X_train, X_valid, X_test, Y_train, Y_valid, Y_test

In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten
from hyperopt import Trials, STATUS_OK, tpe

def model(X_train, X_valid, X_test, Y_train, Y_valid, Y_test):
    model = Sequential()
    model.add(Dense(1024, input_shape=(180,),activation={{choice(['relu','sigmoid','tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([512, 1024, 256])}}, activation={{choice(['relu','sigmoid','tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([256, 1024, 64])}}, activation={{choice(['relu','sigmoid','tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([128, 256, 512])}}, activation={{choice(['relu','sigmoid','tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([64, 256, 512])}}, activation={{choice(['relu','sigmoid','tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([32, 64, 128])}}, activation={{choice(['relu','sigmoid','tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([16, 32, 64])}}, activation={{choice(['relu','sigmoid','tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense(2, activation='softmax'))


    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop(lr={{uniform(0.00003, 0.00006)}},decay=0.0000001),
                  metrics=['accuracy'])

    model.fit(X_train,
              Y_train,
              epochs=epochs,
              batch_size=32,
              validation_data=(X_valid,Y_valid),
              shuffle=True)
    
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [21]:
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=20,
                                          trials=Trials(),
                                          notebook_name='Transaction_Desc_Classifier')
    X_train, Y_train, X_test, Y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, Y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import re
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Dropout, Embedding, Flatten
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from keras.preprocessing.text import Tokenizer
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Dropout, Embedding, Flatten
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

try:
    import networkx as n
except:
    pass

>>> Hyperas s

NameError: name 'd1_nodes' is not defined

In [4]:
import networkx as n
print(n.__version__)

1.11
